# CEO-REQUEST CHALLENGE

> Should Olist remove underperforming sellers from its marketplace?

## Problem statement

To analyse the impact of removing the worse sellers from the marketplace, we can start with a what-if analysis: What would have happened if Olist had never accepted these sellers in the past ? For that:

Step ① Compute, for each `seller_id`, and cumulated since the beginning:
- The `revenues` it brings
- The `costs` associated with all its bad reviews
- The resulting `profits` (revenues - costs)
- The number of `orders` (it will impact overall IT costs)


Step ② We can then sort sellers by increasing profits for Olist, and for each number of seller to remove, compute the financial impact it would have made had they never been accepted on the platform. We may find an optimal number of seller to remove that maximizes Olist's profit margin

In [6]:
# Write down a detailed strategy (steps by step) to create the DataFrame you need for step (1)
# Think about how to re-use or updated what logic you have already coded in your `olist` package

<details>
    <summary>Hints</summary>


Starting from your current `seller().get_training_data()` DataFrame:
- Can you easily transform it to compute Olist's positive `revenue_per_seller`? 
- Can you easily transform it to compute Olist's `cost_of_bad_reviews`?

❓Instead of starting again from scratch, investigate in your source code in `seller.py` how was computed the mean `review_score` per seller. Can you imagine a way to ammend your code so as to compute `cost_of_reviews` in the same process ? 
</details>


## Your turn!

In [5]:
# Keep this notebook tidy, you will present it orally to Olist CEO on Friday afternoon